In [19]:
import practicalSPARQL
import pandas as pd
import requests
import os


In [2]:
root_q = 'queries'
root_d = 'results'

# read login data
login = pd.read_json('config_sparql.json')
ENDPOINT = login['endpoint'][0]
USERNAME = login['username'][0]
PASSWORD = login['password'][0]

# create sparql object
sparql = practicalSPARQL.practicalWrapper(ENDPOINT)
sparql.setCredentials(USERNAME, PASSWORD)

print("--- Querying ENDPOINT: {} ---".format(ENDPOINT))

--- Querying ENDPOINT: http://devmeta.sphaera.mpiwg-berlin.mpg.de/sparql ---


In [14]:
q = practicalSPARQL.stringify_SPARQL('elements_query_w_iiif_230724.sparql')    # select data from the ttl file as a dataframe
df = sparql.select_as_dataframe(q) 
len(df)

21116

In [15]:
df = df.drop_duplicates(subset=['cluster_name'], keep='first')
df

,images,cluster_name,cks,book,bid,part_or_adaption,part_or_adaption_label,custom_identifier,place,year,bb,pagenum,IIIFimage
0,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_00770,CK_Motion of the Eighth Sphere,http://sphaera.mpiwg-berlin.mpg.de/id/item/0be...,1924,http://sphaera.mpiwg-berlin.mpg.de/id/part/1ea...,Theoricae novae planetarum of Peurbach,104,Venice,1488,"xywh=330,247,612,673",134,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
1,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_01419,CK_Motion of the Eighth Sphere,http://sphaera.mpiwg-berlin.mpg.de/id/item/0be...,1924,http://sphaera.mpiwg-berlin.mpg.de/id/part/1ea...,Theoricae novae planetarum of Peurbach,104,Venice,1488,"xywh=68,324,607,634",133,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
2,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_02192,CK_Motion of the Eighth Sphere at Boundary,http://sphaera.mpiwg-berlin.mpg.de/id/item/0be...,1924,http://sphaera.mpiwg-berlin.mpg.de/id/part/1ea...,Theoricae novae planetarum of Peurbach,104,Venice,1488,"xywh=349,321,631,673",136,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
3,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_01120,CK_Complete Motion of the Eighth Sphere,http://sphaera.mpiwg-berlin.mpg.de/id/item/0be...,1924,http://sphaera.mpiwg-berlin.mpg.de/id/part/1ea...,Theoricae novae planetarum of Peurbach,104,Venice,1488,"xywh=325,272,601,684",140,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
4,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_01205,CK_Sun Lines and Motion,http://sphaera.mpiwg-berlin.mpg.de/id/item/0be...,1924,http://sphaera.mpiwg-berlin.mpg.de/id/part/730...,Disputationum contra Cremonensia,103,Venice,1488,"xywh=410,705,303,298",71,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20483,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_02101,CK_Sun Orbit,http://sphaera.mpiwg-berlin.mpg.de/id/item/59f...,1950,http://sphaera.mpiwg-berlin.mpg.de/id/part/678...,Treatise on the Sphere by Rocamora,568,Madrid,1599,"xywh=535,1092,2429,2429",262,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
20494,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_03281,CK_Lunar Phases,http://sphaera.mpiwg-berlin.mpg.de/id/item/59f...,1950,http://sphaera.mpiwg-berlin.mpg.de/id/part/678...,Treatise on the Sphere by Rocamora,568,Madrid,1599,"xywh=236,557,2460,3224",281,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
20497,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_02447,CK_Meridian,http://sphaera.mpiwg-berlin.mpg.de/id/item/59f...,1950,http://sphaera.mpiwg-berlin.mpg.de/id/part/678...,Treatise on the Sphere by Rocamora,568,Madrid,1599,"xywh=325,1550,2260,1141",275,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
20517,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_03364,CK_Positions of the Zenith,http://sphaera.mpiwg-berlin.mpg.de/id/item/6d6...,1752,http://sphaera.mpiwg-berlin.mpg.de/id/part/c61...,Tractatus de sphaera,100,Venice,1601,"xywh=308,331,156,146",21,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...


In [21]:
#download images

for entry in df.iterrows():
    downloadlink = entry[1].IIIFimage
    downloadlink = downloadlink.split("full")
    downloadlink_merged = downloadlink[0] + (entry[1].bb).split("=")[1] + "/!300,300" + downloadlink[1] 
    img_data = requests.get(downloadlink_merged).content
    cks = entry[1].cks.split(", ")
    for ck in cks:
        if not os.path.exists("images_ck/" + ck + "/"):
            os.makedirs("images_ck/" + ck + "/")
        with open("images_ck/" + ck + "/"+ entry[1].cluster_name +".jpg", 'wb') as handler:
            handler.write(img_data)